In [1]:
!git clone https://github.com/Daisybiubiubiu/EEG-Emotion-Recognition.git

Cloning into 'EEG-Emotion-Recognition'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 5218 (delta 23), reused 41 (delta 13), pack-reused 5163
Receiving objects: 100% (5218/5218), 655.44 MiB | 24.60 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Checking out files: 100% (1298/1298), done.


In [0]:
#!ls -R

In [0]:
#!cd ./EEG-Emotion-Recognition  

In [0]:
#!ls

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import time

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,KFold


In [6]:
frameNum = 60 

# load data
dfs = []
for i in range(1,33):
  for j in range(1,41):
    filename = './EEG-Emotion-Recognition/CWT/File_%dframe_exscale/participant%dvideo%d.txt'%(frameNum,i,j)
    cols = [i for i in range(frameNum)]
    df = pd.read_csv(filename, header = None, usecols = cols, delimiter=',')   
    dfs.append(df.values)
    #print('participant%dvideo%d.txt'%(i,j))

dfs = np.array(dfs)
print('dataLoaded:')
print(dfs.shape)

dataLoaded:
(1280, 1024, 60)


In [0]:
# normalize
x_min = dfs.min(axis = (1,2),keepdims=True)
x_max = dfs.max(axis = (1,2),keepdims=True)
dfs_normal = (dfs-x_min)/(x_max-x_min)


In [8]:
depth = 3
# divide frames ,or 60s is too long for a single 3dinput
reshape_dfs = np.split(dfs_normal, frameNum/depth, axis=2)
reshape_dfs = np.array(reshape_dfs)
reshape_dfs = np.reshape(reshape_dfs,[-1,1024,depth])
print(reshape_dfs.shape)

(25600, 1024, 3)


In [9]:
# load label
cols = ['valence', 'arousal', 'dominance', 'liking']
label_df = pd.read_csv('./EEG-Emotion-Recognition/CWT/label.txt',
    usecols = [i for i in range(4)], header=None, delimiter=',' )
print(label_df.shape)
label_df.columns = cols
label_df[label_df<5] = 0
label_df[label_df>=5] = 1


(1280, 4)


In [10]:
# valence
label = label_df['valence'].astype(int).values
label = np.tile(label,20)
print(label.shape)


(25600,)


In [0]:
class cnn_classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv11 = nn.Conv3d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.conv12 = nn.Conv3d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.pool1 = nn.MaxPool3d(kernel_size=2, padding=(0,0,1))
    
    self.conv21 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.conv22 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.pool2 = nn.MaxPool3d(kernel_size=2, padding=(0,0,1))
    
    self.conv31 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
    self.conv32 = nn.Conv3d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
    self.pool3 = nn.MaxPool3d(kernel_size=2, padding=0)
    

    self.fc_layer = nn.Linear(128*4*4*1, 2)
    
    self.dropout_layer = nn.Dropout(p=0.5)

  def forward(self, xb):
    h1 = self.conv11(xb)
    h1 = self.conv12(h1)
    h1 = self.dropout_layer(h1)
    h1 = self.pool1(h1)
    h1 = F.relu(h1)

    h2 = self.conv21(h1)
    h2 = self.conv22(h2)
    #h2 = self.dropout_layer(h2)
    h2 = self.pool2(h2)
    h2 = F.relu(h2) 

    h3 = self.conv31(h2)
    h3 = self.conv32(h3)
    #h3 = self.dropout_layer(h3)
    h3 = self.pool3(h3)
    h3 = F.relu(h3) 
    
    
    # flatten the output from conv layers before feeind it to FC layer
    flatten = h3.view(-1, 128*4*4*1)
    out = self.fc_layer(flatten)
    #out = self.dropout_layer(out)
    return out


In [0]:
def train_model(model, x_train, y_train, x_test, y_test, epochs=50 , batch_size=32, lr=0.0001, weight_decay=0):
  # data
  train_dataset = TensorDataset(x_train, y_train)
  train_data_loader = DataLoader(train_dataset, batch_size=batch_size)

  # loss function
  loss_func = F.cross_entropy

  # optimizer
  #optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

  # figure
  train_a = list([])
  test_a = list([])

  # training loop
  for epoch in range(epochs):
    model.train()
    tic = time.time()
    acc_train = []
    for xb, yb in train_data_loader:    
      xb, yb = xb.to(device), yb.to(device)
      pred = model(xb)
      loss = loss_func(pred, yb)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      acc_train.append(pred.detach().argmax(1).eq(yb).float().mean().cpu().numpy())
    acc_train = np.mean(acc_train)
    toc = time.time()
    
    with torch.no_grad():
      model.eval()
      y_pred = model(x_test.to(device))
      acc = y_pred.argmax(1).eq(y_test.to(device)).float().mean().cpu().numpy()

    train_a.append(acc_train)
    test_a.append(acc)
    print('Loss at epoch %d : %f, train_acc: %f, test_acc: %f, running time: %d'% (epoch, loss.item(), acc_train, acc, toc-tic))
  
  train_amax = max(train_a)
  test_amax = max(test_a)

  # draw an accuray figure
  #plt.plot(train_a,'y.-.')
  #plt.plot(test_a,'.-.')
  #plt.xlabel('epoch')
  #plt.ylabel('accuracy')
  return train_amax,test_amax


In [13]:
# k_folds validation
x_train = reshape_dfs
y_train = label
train_acc = []
test_acc = []
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(x_train, y_train):  
    print(train_index, test_index)

    x_train_fold = x_train[train_index] 
    y_train_fold = y_train[train_index] 
    x_test_fold = x_train[test_index] 
    y_test_fold = y_train[test_index] 
    
    x_train_fold = torch.from_numpy(x_train_fold).float()
    y_train_fold = torch.from_numpy(y_train_fold).long()
    x_test_fold = torch.from_numpy(x_test_fold).float()
    y_test_fold = torch.from_numpy(y_test_fold).long()
 
    print(x_train_fold.shape, y_train_fold.shape) 
    print(x_test_fold.shape, y_test_fold.shape) 

    model = cnn_classifier()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    train_a, test_a = train_model(model, x_train_fold.view(-1, 1, 32, 32, depth), y_train_fold, x_test_fold.view(-1, 1, 32, 32, depth), y_test_fold)
    train_acc.append(train_a)
    test_acc.append(test_a)


print(train_acc)
print(test_acc)

[ 2560  2561  2562 ... 25597 25598 25599] [   0    1    2 ... 2557 2558 2559]
torch.Size([23040, 1024, 3]) torch.Size([23040])
torch.Size([2560, 1024, 3]) torch.Size([2560])
Loss at epoch 0 : 0.691355, train_acc: 0.565625, test_acc: 0.565625, running time: 23
Loss at epoch 1 : 0.691395, train_acc: 0.562934, test_acc: 0.565625, running time: 22
Loss at epoch 2 : 0.692097, train_acc: 0.565625, test_acc: 0.565625, running time: 22
Loss at epoch 3 : 0.692451, train_acc: 0.564410, test_acc: 0.565625, running time: 22
Loss at epoch 4 : 0.786657, train_acc: 0.582075, test_acc: 0.598047, running time: 22
Loss at epoch 5 : 0.757559, train_acc: 0.607248, test_acc: 0.605859, running time: 22
Loss at epoch 6 : 0.790426, train_acc: 0.627604, test_acc: 0.626172, running time: 22
Loss at epoch 7 : 0.716676, train_acc: 0.656250, test_acc: 0.670313, running time: 22
Loss at epoch 8 : 0.613839, train_acc: 0.689714, test_acc: 0.678906, running time: 22
Loss at epoch 9 : 0.547511, train_acc: 0.723915, tes

In [25]:
for i in range(10):
  print(test_acc[i])

0.82109374
0.8167969
0.8625
0.8640625
0.8460938
0.85976565
0.85664064
0.86328125
0.82734376
0.8167969


In [28]:
print(sum(test_acc)/10)

0.8434375107288361
